# Advanced RAG 간단구성 - 쿼리 익스펜션

- 관련 패키지 임포트
- 활용 LLM api정보 설정
- RAG 파이프라인 글로벌 세팅 설정
- 기준 데이터셋 로드
- 기준 데이터셋 전처리
- 벡터스토어인덱스 설정
- 베이스 쿼리 엔진 설정
- 어드벤스드 쿼리 엔진(서브쿼리엔진) 설정

In [1]:
# 관련 패키지 임포트
import os
from datasets import load_dataset 
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core import Document
from llama_index.core import VectorStoreIndex
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
import nest_asyncio

nest_asyncio.apply()

In [2]:
# RAG 파이프라인 글로벌 설정
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)

Settings.llm=OpenAI(model='gpt-3.5-turbo',temperature=0)

In [3]:
# 기준 데이터셋 로드 - 
ds = load_dataset("lcw99/wikipedia-korean-20240501", split='train[-10:]')

In [4]:
# 기준 데이터셋 전처리
data = ds.to_pandas()[['title', 'text']]
data.drop_duplicates(subset='text', keep='first', inplace=True)
data.head()

,title,text
0,학도요,"'''학도요'''(, )는 도요목 도요과에 속하는 나그네새이다.\n부리와 다리가 길..."
1,뒷부리도요,"'''뒷부리도요'''(, )는 도요목 도요과에 속하는 나그네새이다.\n부리가 길고 ..."
2,칼 스벤손,"'''칼 빌헬름 스벤손'''(, 1984년 3월 21일, 옌셰핑 주 옌셰핑 ~)은 ..."
3,노랑발도요,"'''노랑발도요'''(, )는 도요목 도요과에 속하는 나그네새이다.\n다리는 비교적..."
4,신창균,"'''신창균'''(申昌均, 1908년~2003년 3월 5일)은 대한민국의 독립운동이..."


In [5]:
# 데이터셋 라마인덱스 다큐먼트 오브젝트로 변환
docs = []

for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        extra_info={'title': row['title']}
    ))

In [6]:
docs

[Document(id_='85aab55c-fda7-459e-bdc4-f9d2999e0a9d', embedding=None, metadata={'title': '학도요'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text="'''학도요'''(, )는 도요목 도요과에 속하는 나그네새이다.\n부리와 다리가 길고 아랫부리의 절반이 붉은색이다. \n=== 여름깃 ===\n전체적으로 검은색이고 몸윗면에 흰 반점이 흩어져 있다. \n=== 겨울깃 ===\n전체적으로 회갈색이고 흰색 눈썹선이 명확하다. 몸아랫면은 흰색이며 가슴과 옆구리에 엷은 회갈색 기운이 있다.\n무논, 습지, 하구, 갯벌 등에서 무리지어 생활한다. 유라시아대륙 북부에서 번식하고 유럽 남부, 아프리카, 인도, 동남아시아에서 월동한다.\n분류:도요과\n분류:한국의 새\n분류:한국의 나그네새\n분류:유럽의 새\n분류:아프리카의 새\n분류:인도의 새\n분류:동남아시아의 새", path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 Document(id_='a8b6cd5b-e015-4e60-a7a2-52beb77a2215', embedding=None, metadata={'title': '뒷부리도요'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {

In [7]:
# 벡터스토어 인덱스 설정
vector_index = VectorStoreIndex.from_documents(
    docs,
    use_asnyc=True
    )

In [8]:
# 쿼리 엔진 설정
vector_query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [9]:
# 어드벤스트 쿼리 엔진(서브퀘스천쿼리엔진)
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="wikipedia",
            description="사용자 질문에 대답할 때 사용",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

- 구축된 Naive-RAG 파이프라인 이용한 기본적인 Q&A 테스팅

In [10]:
response = query_engine.query('칼 스벤손이 누구야?')

Generated 1 sub questions.
[wikipedia] Q: Who is Karl Svenonius?
[wikipedia] A: Karl Svenonius is a former professional football player from Sweden who played as a central defender for clubs such as Yenshaping Shidra, IFK, Rangers, and Caen. He also made one appearance for the Sweden national team and participated in the 2006 World Cup representing his country.


In [11]:
# 검색에 대한 답변
response.response

'Karl Svenonius는 스웨덴 출신의 전직 프로 축구 선수로, Yenshaping Shidra, IFK, Rangers, Caen 등의 클럽에서 중앙 수비수로 활약했습니다. 또한 스웨덴 대표팀에서 한 번 출전하였으며, 2006년 월드컵에 출전하여 국가를 대표했습니다.'

In [13]:
# 소스노드 구조
response.source_nodes

[NodeWithScore(node=TextNode(id_='86cfa888-dfc6-4fd3-b33b-0501054dd847', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Sub question: Who is Karl Svenonius?\nResponse: Karl Svenonius is a former professional football player from Sweden who played as a central defender for clubs such as Yenshaping Shidra, IFK, Rangers, and Caen. He also made one appearance for the Sweden national team and participated in the 2006 World Cup representing his country.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None),
 NodeWithScore(node=TextNode(id_='00f8e70e-32f1-4c3c-aa20-c710683d16f5', embedding=None, metadata={'title': '칼 스벤손'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3d919858-15c2-

In [12]:
print(response.source_nodes[1].score)
print(response.source_nodes[2].score)

0.22168312995591083
0.20515683552526848


In [14]:
# 참조 청크의 사이즈(글자수)
print(response.source_nodes[1].node.start_char_idx)
print(response.source_nodes[1].node.end_char_idx)
print(response.source_nodes[1].node.end_char_idx-response.source_nodes[1].node.start_char_idx)

0
950
950


In [15]:
# 다른 질문
response = query_engine.query('칼 스벤손이 언제 태어났고, 어떤 월드컵에 참가했지?')

Generated 2 sub questions.
[wikipedia] Q: When was Karl Svensson born?
[wikipedia] Q: Which World Cup did Karl Svensson participate in?
[wikipedia] A: Karl Svensson was born in 1984.
[wikipedia] A: Karl Svensson participated in the 2006 World Cup.


In [16]:
response.response

'Karl Svensson was born in 1984 and participated in the 2006 World Cup.'

In [17]:
# 소스노드 구조
response.source_nodes

[NodeWithScore(node=TextNode(id_='0736ff86-5bb7-4dc7-88a7-d85156091d57', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Sub question: When was Karl Svensson born?\nResponse: Karl Svensson was born in 1984.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None),
 NodeWithScore(node=TextNode(id_='3d01b75e-0e4b-4fd5-97a0-7b5792c599c7', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Sub question: Which World Cup did Karl Svensson participate in?\nResponse: Karl Svensson participated in the 2006 World Cup.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), sc

In [18]:
docs

[Document(id_='85aab55c-fda7-459e-bdc4-f9d2999e0a9d', embedding=None, metadata={'title': '학도요'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text="'''학도요'''(, )는 도요목 도요과에 속하는 나그네새이다.\n부리와 다리가 길고 아랫부리의 절반이 붉은색이다. \n=== 여름깃 ===\n전체적으로 검은색이고 몸윗면에 흰 반점이 흩어져 있다. \n=== 겨울깃 ===\n전체적으로 회갈색이고 흰색 눈썹선이 명확하다. 몸아랫면은 흰색이며 가슴과 옆구리에 엷은 회갈색 기운이 있다.\n무논, 습지, 하구, 갯벌 등에서 무리지어 생활한다. 유라시아대륙 북부에서 번식하고 유럽 남부, 아프리카, 인도, 동남아시아에서 월동한다.\n분류:도요과\n분류:한국의 새\n분류:한국의 나그네새\n분류:유럽의 새\n분류:아프리카의 새\n분류:인도의 새\n분류:동남아시아의 새", path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 Document(id_='a8b6cd5b-e015-4e60-a7a2-52beb77a2215', embedding=None, metadata={'title': '뒷부리도요'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {

In [19]:
# 다른 질문 - naive engine
response = vector_query_engine.query('신창균과 칼 스벤손에 대해서 알려줘')

In [20]:
response.response

'신창균과 칼 스벤손은 모두 축구 선수이다. 신창균에 대한 추가 정보는 제공되지 않았으나, 칼 스벤손은 스웨덴의 전직 프로 축구 선수로, 중앙 수비수로 활약했으며 스웨덴 국가대표팀에도 출전한 경력이 있다. 스벤손은 IFK, 레인저스, 캉 등에서 뛰었으며 프랑스와 스웨덴을 넘나들며 활약한 선수이다.'

In [21]:
# 다른 질문 - query expander
response = query_engine.query('신창균과 칼 스벤손에 대해서 알려줘')

Generated 2 sub questions.
[wikipedia] Q: 신창균에 대한 정보
[wikipedia] Q: 칼 스벤손에 대한 정보
[wikipedia] A: 신창균은 대한민국의 독립운동, 민주화운동, 그리고 통일운동에 참여한 인물로, 1908년 충청북도 영동에서 태어났습니다. 어린 나이에 3.1운동에 참가하다가 체포되었지만 매만 맞고 풀려났으며, 이후 중국으로 망명하여 대한민국 임시정부와 협력하였습니다. 한국독립당의 활동가로 활발히 활동하였고, 북측의 단독정부 수립을 막기 위해 김일성과 회담도 가졌습니다. 그 이후에는 진보당사건 및 5.16군사정변에 연루되어 수감생활을 하였고, 풀려난 뒤에는 군사독재 종식을 위한 민주화운동 및 통일운동에 헌신하였습니다.
[wikipedia] A: 칼 빌헬름 스벤손은 스웨덴의 전직 프로 축구 선수로, 중앙 수비수로 활약했으며, 옌셰핑스 셰드라, IFK, 레인저스, 캉에서 뛰었습니다. 그는 스웨덴 국가대표팀에 1번 출전했고, 2006년 월드컵에 출전한 경력을 가지고 있습니다. 스벤손은 2014년에 현역에서 은퇴했으며, U-17, U-19, 그리고 U-21 연령대의 스웨덴 국가대표팀에서도 활약했습니다.


In [22]:
response.response

'신창균은 대한민국의 독립운동, 민주화운동, 그리고 통일운동에 참여한 인물로, 1908년 충청북도 영동에서 태어났습니다. 어린 나이에 3.1운동에 참가하다가 체포되었지만 매만 맞고 풀려났으며, 이후 중국으로 망명하여 대한민국 임시정부와 협력하였습니다. 한국독립당의 활동가로 활발히 활동하였고, 북측의 단독정부 수립을 막기 위해 김일성과 회담도 가졌습니다. 그 이후에는 진보당사건 및 5.16군사정변에 연루되어 수감생활을 하였고, 풀려난 뒤에는 군사독재 종식을 위한 민주화운동 및 통일운동에 헌신하였습니다.\n\n칼 빌헬름 스벤손은 스웨덴의 전직 프로 축구 선수로, 중앙 수비수로 활약했으며, 옌셰핑스 셰드라, IFK, 레인저스, 캉에서 뛰었습니다. 그는 스웨덴 국가대표팀에 1번 출전했고, 2006년 월드컵에 출전한 경력을 가지고 있습니다. 스벤손은 2014년에 현역에서 은퇴했으며, U-17, U-19, 그리고 U-21 연령대의 스웨덴 국가대표팀에서도 활약했습니다.'